# TODO:
* POS-TAGGER is unreliable


    ADJ: adjective
    ADP: adposition
    ADV: adverb
    AUX: auxiliary verb
    CONJ: coordinating conjunction
    DET: determiner
    INTJ: interjection
    NOUN: noun
    NUM: numeral
    PART: particle
    PRON: pronoun
    PROPN: proper noun
    PUNCT: punctuation
    SCONJ: subordinating conjunction
    SYM: symbol
    VERB: verb
    X: other

* Vale a pena português?

In [18]:
import textacy
import logging
import pandas as pd
import itertools
import math
from collections import Counter

LOGGER = logging.getLogger(__name__)

def make_stats_from_sents(doc):
    
    a = [Counter(map(lambda w_t: w_t[1], tagged_sen)) for tagged_sen in doc.pos_tagged_text]
    b = pd.DataFrame(a).fillna(0).agg(['mean', 'std'])
    c = b.unstack()
    n_labels = c.index.map(lambda l: 'sents_{}_{}'.format(*l))
    c.index = n_labels
    
    return c

# TODO: use hierarchical multiindex to group related metrics, like basic metrics and pos metrics
def make_stats(text, name):
    doc = textacy.Doc(text, lang="pt")

    stats = textacy.text_stats.TextStats(doc)

    # POS counter for all the text
    c = Counter([tag for (w, tag) in itertools.chain.from_iterable(doc.pos_tagged_text)])
    
    df = pd.Series(dict(stats.basic_counts, **c))
    
    # POS counter statistics for sentences
    ss = make_stats_from_sents(doc)
    df = pd.concat([df, ss])
    
    # Readability Metrics
    df['easy_flesch_reading_ease'] = (-84.6 * stats.n_syllables / stats.n_words) - (1.015 * stats.n_words / stats.n_sents) + 206.835
    df['diff_flesch_kincaid_grade_leve'] = (11.8 * stats.n_syllables / stats.n_words) + (0.39 * stats.n_words / stats.n_sents) - 15.59
    if stats.n_sents < 30:
        LOGGER.warning('SMOG score may be unreliable for n_sents < 30')
    df['diff_smog_index'] = (1.0430 * math.sqrt(30 * stats.n_polysyllable_words / stats.n_sents)) + 3.1291
    df['diff_gunning_fog_index'] = 0.4 * ((stats.n_words / stats.n_sents) + (100 * stats.n_polysyllable_words / stats.n_words))
    df['diff_coleman_liau_index'] = (5.879851 * stats.n_chars / stats.n_words) - (29.587280 * stats.n_sents / stats.n_words) - 15.800804
    df['diff_automated_readability_index'] = (4.71 * stats.n_chars / stats.n_words) + (0.5 * stats.n_words / stats.n_sents) - 21.43
    df['diff_lix'] = (stats.n_words / stats.n_sents) + (100 * stats.n_long_words / stats.n_words)
    df['diff_gulpease_index'] = (300 * stats.n_sents / stats.n_words) - (10 * stats.n_chars / stats.n_words) + 89
    # TODO: wiener_sachtextformel
    
    df.name = name
    
    return df.fillna(0)

In [19]:
# source: https://www2.uol.com.br/sciam/noticias/cientistas_implantam_cerebros_humanos_em_miniatura_em_cranios_de_camundongos.html
s1 = """Mas a verossimilhança dos organoides cerebrais humanos é limitada: quando crescem mais do que alguns milímetros, o oxigênio e os nutrientes não chegam às células mais internas. “Em nossas mãos, os organoides pararam de crescer após mais ou menos cinco semanas” diz Fred Gage, que liderou o estudo. “É um problema ligado ao tamanho,  não à idade. Algumas células morrem mesmo no pico de criação do organoide, a partir da décima semana. E isso piora com o tempo."""
# source: http://chc.org.br/pequenos-notaveis/
s2 = """Esses peixes são praticamente invisíveis aos nossos olhos por serem bastante pequenos, com poucos centímetros de comprimento. Por isso, encontrar um deles em um recife de coral é praticamente achar uma agulha em um palheiro! Com um tamanho tão pequeno e vivendo em um ambiente repleto de predadores, a maioria dos peixes criptobênticos vive em esconderijos: em pequenas cavidades dos recifes, dentro de esponjas, entre os ramos de lírios e ouriços-do-mar, corais, anêmonas…"""
# source: https://www.opovo.com.br/esportes/futebol/times/ceara/2018/04/ceara-solicita-imagens-da-arena-castelao-para-identificar-agressor-de.html
s3 = """"Foi feito um pedido informal, pois precisamos saber o local exato da agressão, para entrarmos com um ofício pedindo as imagens do ambiente", disse Jamilson em entrevista ao O POVO. O Setor Premium, área que Mari foi agredida verbal e moralmente, tendo seu braço puxado quando tentou sair da área, é dividido em quatro partes: área dos banheiros, conveniência, corredor de transição e arquibancada com capacidade para 4.200 pessoas."""
# source: http://ufc.br/noticias/noticias-de-2018/11133-nota-sobre-a-acao-civil-publica-n-0805469-35-2018-4-05-8100
s4 = """A Universidade Federal do Ceará manifesta estranhamento diante do fato de a Procuradoria-Geral da República haver divulgado na imprensa e nas mídias sociais, desde a manhã desta quinta-feira (26), informações referentes à Ação Civil Pública nº 0805469-35.2018.4.05.8100, a qual tem por objeto discussão sobre a disciplina optativa O Golpe de 2016: o Futuro da Democracia, que, sob a autonomia universitária, inaugura debate acerca de fatos recentes de nossa história. A divulgação ocorreu antes mesmo do protocolo da ação na Justiça Federal no Ceará, o que se deu somente às 10h57min da presente data, não tendo sido a UFC notificada até o momento."""

In [20]:
len(s1), len(s2), len(s3), len(s4)

(460, 473, 432, 648)

In [21]:
pd.concat([
    make_stats(s1, 'scientific'), 
    make_stats(s2, 'child'),
    make_stats(s3, 'opovo'),
    make_stats(s4, 'ufc')
], axis=1).fillna(0)

SMOG score may be unreliable for n_sents < 30
SMOG score may be unreliable for n_sents < 30
SMOG score may be unreliable for n_sents < 30
SMOG score may be unreliable for n_sents < 30


,scientific,child,opovo,ufc
diff_automated_readability_index,6.902667,11.928506,19.687286,19.777259
diff_coleman_liau_index,9.027465,12.290535,12.837098,12.549094
diff_flesch_kincaid_grade_leve,12.472500,15.977240,21.660000,22.691963
diff_gulpease_index,65.500000,54.194805,47.428571,47.785047
diff_gunning_fog_index,17.833333,21.206494,24.857143,27.724611
diff_lix,40.833333,50.418831,63.571429,66.507788
diff_smog_index,14.790195,17.693802,20.736967,22.918634
easy_flesch_reading_ease,29.389167,14.800146,2.110000,-4.101246
n_chars,368.000000,388.000000,351.000000,531.000000
n_long_words,22.000000,24.000000,20.000000,33.000000


# Clarice x Machado

In [310]:
import codecs 

with codecs.open('../data/perto_do_coracao_selvagem.txt', 'r', 'utf-8') as f:
    clarice = f.read()
    
with codecs.open('../data/dom_casmurro.txt', 'r', 'utf-8') as f:
    machado = f.read()

In [311]:
len(clarice), len(machado)

(291114, 375295)

In [323]:
df = pd.concat([make_stats(clarice, 'Clarice Lispector'), make_stats(machado, 'Machado de Assis')], axis=1)
df

,Clarice Lispector,Machado de Assis
ADJ,3255.000000,3183.000000
ADP,5790.000000,7375.000000
ADV,4888.000000,5961.000000
AUX,670.000000,1128.000000
CCONJ,1586.000000,2779.000000
DET,6581.000000,9113.000000
INTJ,68.000000,68.000000
NOUN,9288.000000,12048.000000
NUM,188.000000,493.000000
PART,2.000000,NaN


In [331]:
df_sent = df.loc[df.index[df.index.str.startswith('sent')].tolist(), :]


    ADJ: adjective
    ADP: adposition
    ADV: adverb
    AUX: auxiliary verb
    CONJ: coordinating conjunction
    DET: determiner
    INTJ: interjection
    NOUN: noun
    NUM: numeral
    PART: particle
    PRON: pronoun
    PROPN: proper noun
    PUNCT: punctuation
    SCONJ: subordinating conjunction
    SYM: symbol
    VERB: verb
    X: other


In [335]:
(df_sent['Clarice Lispector'] - df_sent['Machado de Assis'])\
.sort_values(ascending=False)\
.apply(lambda v: '{:.2f} %'.format(v*100))

sents_SPACE_mean     51.76 %
sents_SPACE_std      31.68 %
sents_ADJ_mean       18.79 %
sents_ADJ_std        18.18 %
sents_ADV_mean        8.15 %
sents_ADV_std         6.69 %
sents_ADP_mean        4.33 %
sents_X_std           4.01 %
sents_NOUN_mean       3.25 %
sents_INTJ_std        2.27 %
sents_X_mean          1.87 %
sents_INTJ_mean       0.37 %
sents_NOUN_std       -0.05 %
sents_ADP_std        -1.31 %
sents_PRON_std       -2.00 %
sents_SCONJ_std      -3.51 %
sents_SYM_mean       -3.80 %
sents_AUX_mean       -4.16 %
sents_NUM_mean       -4.16 %
sents_SCONJ_mean     -4.88 %
sents_AUX_std        -5.06 %
sents_PUNCT_mean     -5.75 %
sents_PRON_mean      -5.86 %
sents_VERB_mean      -6.05 %
sents_DET_mean       -7.49 %
sents_VERB_std       -7.61 %
sents_SYM_std        -8.07 %
sents_NUM_std       -11.29 %
sents_CCONJ_mean    -11.70 %
sents_CCONJ_std     -13.28 %
sents_DET_std       -15.41 %
sents_PUNCT_std     -20.47 %
sents_PROPN_mean    -28.68 %
sents_PROPN_std     -38.92 %
sents_PART_mea

# Experiments from RDF2PT

In [336]:
human = "Albert Einstein era um cientista, que trabalhava na área de Física. Era conhecido pela fórmula de equivalência entre massa e energia. Formou-se na Universidade de Zurique. Einstein ganhou a medalha Max Planck por seu trabalho. Em Princeton, onde morreu, teve sob sua orientação Ernst Gabor Straus."
baseline = "Albert Einstein é cientista, Albert Einstein campo é física, Albert Einstein lugar falecimento Princeton. Albert Einstein ex-instituição é Universidade Zurique, Albert Einstein é conhecido Equivalência massa-energia, Albert Einstein prêmio é Medalha Max Planck, Albert Einstein estudante doutorado é Ernst Gabor Straus."
rdf2pt = "Albert Einstein foi um cientista, o campo dele foi a física e ele faleceu no Princeton. Além disso, sua ex-instituição foi a Universidade de Zurique, ele é conhecido pela Equivalência massa-energia, o prêmio dele foi a Medalha Max Planck e o estudante de doutorado dele foi o Ernst Gabor Straus."

In [339]:
print(human)

Albert Einstein era um cientista, que trabalhava na área de Física. Era conhecido pela fórmula de equivalência entre massa e energia. Formou-se na Universidade de Zurique. Einstein ganhou a medalha Max Planck por seu trabalho. Em Princeton, onde morreu, teve sob sua orientação Ernst Gabor Straus.


In [340]:
print(baseline)

Albert Einstein é cientista, Albert Einstein campo é física, Albert Einstein lugar falecimento Princeton. Albert Einstein ex-instituição é Universidade Zurique, Albert Einstein é conhecido Equivalência massa-energia, Albert Einstein prêmio é Medalha Max Planck, Albert Einstein estudante doutorado é Ernst Gabor Straus.


In [341]:
print(rdf2pt)

Albert Einstein foi um cientista, o campo dele foi a física e ele faleceu no Princeton. Além disso, sua ex-instituição foi a Universidade de Zurique, ele é conhecido pela Equivalência massa-energia, o prêmio dele foi a Medalha Max Planck e o estudante de doutorado dele foi o Ernst Gabor Straus.


In [380]:
import spacy

nlp = spacy.load('pt')

In [395]:
t = """

Peter Yuen, chefe da polícia, falou aos jornalistas no fim da tarde e confirmou as mortes: “São 9 pessoas mortas e 16 feridas. Recebemos várias ligações de pessoas dizendo que um veículo estava atropelando pessoas por volta de 13h30. Localizamos a van e o motorista está sob custódia".

De acordo com a agência Reuters, o hospital Sunnybrook recebeu 8 pacientes vindos do incidente. Das oito pessoas admitidas no hospital, uma morreu, 5 estão em estado em estado crítico, 1 pessoa em estado grave e 1 pessoa em bom estado.
"""

In [396]:
d = nlp(t)

In [397]:
for ent in d.ents:
    print("{} {}".format(ent.text, ent.label_))

Peter Yuen PER
São 9 MISC
Localizamos PER
agência Reuters ORG
Sunnybrook ORG


In [365]:
import seaborn as sns
cm = sns.diverging_palette(255, 0, l=60, n=3, center="light", as_cmap=True)

df = pd.concat([make_stats(human, 'human'), make_stats(baseline, 'baseline'), make_stats(rdf2pt, "rdf2pt")], axis=1)
df.fillna(0).style.background_gradient(cmap=cm, axis=1)

SMOG score may be unreliable for n_sents < 30
SMOG score may be unreliable for n_sents < 30
SMOG score may be unreliable for n_sents < 30


,human,baseline,rdf2pt
ADJ,0,4,2
ADP,8,0,7
ADV,1,1,1
AUX,0,1,1
CCONJ,1,0,2
DET,6,0,9
NOUN,9,5,6
PRON,1,0,8
PROPN,12,26,12
PUNCT,9,9,8
